In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os
import google.generativeai as genai
import json
import time
import sqlite3
load_dotenv()
from keylog_analysis_helper import *
from gemini_prompts import *
import difflib

Notes
- Code reconstruction may be incorrect, double check again
- Measures of progress:
    - Whether code compiles,
    - Ask llm if it progressed?
    - '# characters written
        - Those that are code and not (ask llm)


- does this portion make sense
- does this portion make sense to the solution 


Define model

In [ ]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 1,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-pro-002",
  generation_config=generation_config,
  system_instruction= system_instructions,
)

# chat_session = model.start_chat(
#   history=[
#   ]
# )

In [20]:
a = model.generate_content("hello")

In [21]:
a.parts[0].text

'How can I help you today?\n'

In [31]:
a.usage_metadata.total_token_count

10

Save dataframes with diff

In [ ]:
def form_context(file1_name, file1_prev, file1_current, file1_diff, file2_name, file2_prev, file2_current, file2_diff, instructions):

    context = f"""
        You will help me determine whether a student has made progress towards completing their coding assignment.
        I will provide you with the code from the previous snapshot and the current snapshot. I will also provide you with the diff between the two files.
        
        Here is the previous snapshot for the file named {file1_name} : {file1_prev}
        Here is the current snapshot for the file named {file1_name} : {file1_current}
        Here is the diff between the two files:
        {file1_diff}

        
        Here is the previous snapshot for the file named {file2_name} : {file2_prev}
        Here is the current snapshot for the file named {file2_name} : {file2_current}
        Here is the diff between the two files:
        {file2_diff}

        The instructions for this assignment were as follows {instructions}
        """

In [ ]:
def form_prompt(file1_name, file1_prev, file1_current, file1_diff, file2_name, file2_prev, file2_current, file2_diff, instructions):

    prompt = f"""
    You will help me determine whether a student has made progress towards completing their coding assignment.
    I will provide you with the code from the previous snapshot and the current snapshot. I will also provide you with the diff between the two files.
    
    Here is the previous snapshot for the file named {file1_name} : {file1_prev}
    Here is the current snapshot for the file named {file1_name} : {file1_current}
    Here is the diff between the two files:
    {file1_diff}

    
    Here is the previous snapshot for the file named {file2_name} : {file2_prev}
    Here is the current snapshot for the file named {file2_name} : {file2_current}
    Here is the diff between the two files:
    {file2_diff}

    The instructions for this assignment were as follows {instructions}
    """

    return prompt


    # The autograder code that is used to score the student is as follows {}

In [27]:
instructions_a3 = read_file_as_string("archive/CompleteAssignments/a3_P0_c0/Project3.txt")
instructions_a4 = read_file_as_string("archive/CompleteAssignments/a4_P0_c0/Project4.txt")

Define Loop

In [37]:
differ = difflib.Differ()

def make_llm_comparisons(a3=True):

    # Read df based on assignment type
    if a3:
        df = pd.read_csv("KeylogDatasets/a3.csv")
        # df.drop(columns=["Unnamed: 0", "autograder_error","bustersAgents.py_length","inference.py_length"], inplace=True)
    else:
        df = pd.read_csv("KeylogDatasets/a4.csv")
        # df.drop(columns=["Unnamed: 0","autograder_error","NeuralNet.py_length","NeuralNetUtil.py_length"], inplace=True)
   
    df["Prediction"] = None
    df['Diff'] = None
    p_list = p_list_a3_log_analysis if a3 else p_list_a4_log_analysis
    assignment = 'a3' if a3 else 'a4'
    snapshots_dir = snapshots_dir_a3 if a3 else snapshots_dir_a4

    total_token = 0
    for p in p_list[:1]:
        print("--Now working on participant", p)
        df_temp = df[df["participant"] == p]

        for idx, row in df_temp.iterrows(): # start from snapshot 1
            if row['snapshot'] == 0:
                continue
            # Read files
            current_snapshot = row['snapshot']
            prev_snapshot = row['snapshot'] - 1
            print("Now comparing snapshots", prev_snapshot, current_snapshot)
            
            if a3:
                prev_snapshot_dir = os.path.join(snapshots_dir, f'{assignment}_{p}_c{current_snapshot}', "tracking")
                current_snapshot_dir = os.path.join(snapshots_dir, f'{assignment}_{p}_c{prev_snapshot}', "tracking")
                file1_prev = read_file_as_string(os.path.join(prev_snapshot_dir, "bustersAgents.py"))
                file1_current = read_file_as_string(os.path.join(current_snapshot_dir, "bustersAgents.py"))
                file2_prev = read_file_as_string(os.path.join(prev_snapshot_dir, "inference.py"))
                file2_current = read_file_as_string(os.path.join(current_snapshot_dir, "inference.py"))
            else:
                prev_snapshot_dir = os.path.join(snapshots_dir, f'{assignment}_{p}_c{prev_snapshot}')
                current_snapshot_dir = os.path.join(snapshots_dir, f'{assignment}_{p}_c{current_snapshot}')
                file1_prev = read_file_as_string(os.path.join(prev_snapshot_dir, "NeuralNet.py"))
                file1_current = read_file_as_string(os.path.join(current_snapshot_dir, "NeuralNet.py"))
                file2_prev = read_file_as_string(os.path.join(prev_snapshot_dir, "NeuralNetUtil.py"))
                file2_current = read_file_as_string(os.path.join(current_snapshot_dir, "NeuralNetUtil.py"))

            # Run diff
            file1_diff = differ.compare(file1_prev, file1_current)
            file2_diff = differ.compare(file2_prev, file2_current)
            
            # Read autograder code contents
            if a3:
                autograder_code = None
                assignment_instructions = None
                instructions = instructions_a3
                file1_name = "bustersAgents.py"
                file2_name = "inference.py"
            else:
                autograder_code = None
                assignment_instructions = None
                instructions = instructions_a4
                file1_name = "NeuralNet.py"
                file2_name = "NeuralNetUtil.py"

            # Ask the llm
            prompt = form_prompt(file1_name, file1_prev, file1_current, file1_diff, file2_name, file2_prev, file2_current, file2_diff, instructions)
            print (f"Prompt of length {len(prompt)}")
            # return prompt 
            

            try:
                response = model.generate_content(prompt)
                total_token += response.usage_metadata.total_token_count
                response = response.parts[0].text
            except Exception as e:
                print("Gemini Exception: ", e)
                response = None
    
            print("Response is ", response)
            df.at[idx, "Prediction"] = response
            # df.to_csv("KeylogDatasets/a3_llm.csv", index=False) if a3 else df.to_csv("KeylogDatasets/a4.csv", index=False)
            time.sleep(1)
            print("Total tokens used: ", total_token)
            
        return df

In [38]:
df_a3 = make_llm_comparisons(a3=True)

--Now working on participant P3
Now comparing snapshots 0 1
Prompt of length 74808
Response is  1

Total tokens used:  17692
Now comparing snapshots 1 2
Prompt of length 75191
Response is  1

Total tokens used:  35475
Now comparing snapshots 2 3
Prompt of length 75616
Response is  1

Total tokens used:  53355
Now comparing snapshots 3 4
Prompt of length 75796
Response is  1

Total tokens used:  71273
Now comparing snapshots 4 5
Prompt of length 75798
Response is  1

Total tokens used:  89189
Now comparing snapshots 5 6
Prompt of length 75819
Response is  1

Total tokens used:  107108
Now comparing snapshots 6 7
Prompt of length 75989
Response is  1

Total tokens used:  125064
Now comparing snapshots 7 8
Prompt of length 76162
Response is  1

Total tokens used:  143052
Now comparing snapshots 8 9
Prompt of length 76264
Response is  1

Total tokens used:  161066
Now comparing snapshots 9 10
Prompt of length 76499
Response is  1

Total tokens used:  179130
Now comparing snapshots 10 11
Pr

In [36]:
df_a3